In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import glob
import wandb
import os
import torch.optim as optimizers
import pandas as pd
from copy import deepcopy
from ml_collections import ConfigDict 
import tqdm
import torch.nn.functional as F
import torch
import torch.nn as nn
from collections import defaultdict
import yaml
from sklearn.metrics import roc_auc_score
import networkx as nx

import sys
sys.path.append("../../src")
import dfs_code 
from dfs_transformer import collate_downstream, DFSCodeSeq2SeqFC, TrainerGNN
#torch.multiprocessing.set_sharing_strategy('file_system')

import torch_geometric.nn as tnn
import networkx as nx

2022-02-02 10:21:32.786244: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/cuda/extras/CUPTI/lib64/:/opt/intel/lib:/opt/intel/mkl/lib/intel64:/opt/intel:/opt/ibm/ILOG/CPLEX_Studio1210/cplex/bin/x86-64_linux:/opt/ibm/ILOG/CPLEX_Studio1210/cplex/python/3.7/x86-64_linux:/opt/intel/clck_latest/lib:/opt/intel/daal/lib:/opt/intel/intelpython3/lib:/opt/intel/ipp/lib:/opt/intel/itac_2019/lib:/opt/intel/itac_latest/lib:/opt/intel/mkl/lib:/opt/intel/mkl_/lib:/opt/intel/mpirt/lib:/opt/intel/tbb/lib:/opt/intel/clck/2019.0/lib:/opt/intel/compilers_and_libraries_2019/linux/lib:/opt/intel/compilers_and_libraries/linux/lib:/opt/intel/itac/2019.0.018/lib:/opt/intel/itac_2019/intel64/lib:/opt/intel/itac_latest/intel64/lib:/opt/intel/parallel_studio_xe_2019.0.045/clck_2019/lib:/opt/intel/parallel_studio_xe_2019.0.045/itac_2019/l

In [3]:
config = wandb.config
config.graph_file = "/mnt/ssd/datasets/graphs/reddit_threads/reddit_edges.json"
config.label_file = "/mnt/ssd/datasets/graphs/reddit_threads/reddit_target.csv"
config.batch_size = 50
config.n_epochs = 100
config.learning_rate = 0.01
config.rep = 1
config.max_edges = 1000
config.n_samples = None
config.n_channels = 32
config.n_layers = 2
config.model = "tnn.models.GCN"
config.readout = "tnn.global_mean_pool"
config.num_workers = 5
config.training = {}

In [4]:
run = wandb.init(mode="online", project="karateclub-reddit-50", entity="dfstransformer", 
                 config=config, job_type="evaluation")

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: chrisxx (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2022-02-02 10:21:36.000647: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/cuda/extras/CUPTI/lib64/:/opt/intel/lib:/opt/intel/mkl/lib/intel64:/opt/intel:/opt/ibm/ILOG/CPLEX_Studio1210/cplex/bin/x86-64_linux:/opt/ibm/ILOG/CPLEX_Studio1210/cplex/python/3.7/x86-64_linux:/opt/intel/clck_latest/lib:/opt/intel/daal/lib:/opt/intel/intelpython3/lib:/opt/intel/ipp/lib:/opt/intel/itac_2019/lib:/opt/intel/itac_latest/lib:/opt/intel/mkl/lib:/opt/intel/mkl_/lib:/opt/intel/mpir

In [5]:
with open(config.graph_file, 'r') as f:
    graph_dict = json.load(f)
label_df = pd.read_csv(config.label_file)

In [6]:
def graph2labelledgraph(graph, use_dummy=False):
    graph = deepcopy(graph)
    node_ids = np.unique(graph).tolist()
    dummy = max(node_ids) + 1
    
    edge_labels = len(graph)*[0]
    edgeindex = []
    for e in graph:
        edgeindex += [e]
        edgeindex += [[e[1], e[0]]]
        edge_labels += [0]
    
    if use_dummy:
        for idx in node_ids:
            edgeindex += [[idx, dummy], [dummy, idx]]
            edge_labels += [1, 1]
        node_ids += [dummy]
    
    edgeindex = np.asarray(edgeindex).T
    
    node_labels = []
    for idx in node_ids:
        node_labels += [(edgeindex[0] == idx).sum()]
    return edgeindex, node_labels, edge_labels

In [7]:
from torch_geometric.data import Data, Dataset
from torch_geometric.loader import DataLoader

class KarateClubDataset(Dataset):
    def __init__(self, graph_file, label_file, max_n = None, max_edges=50):
        super().__init__()
        self.graph_file = graph_file
        self.label_file = label_file
        with open(graph_file, 'r') as f:
            self.graph_dict = json.load(f)
        self.label_df = pd.read_csv(label_file)
        self.data = []
        self.maxn = max_n
        self.max_edges = max_edges
        self.preprocess()
    
    def preprocess(self):
        maxn = self.maxn
        maxdegree = 0
        edgeindex_list = []
        vlabels_list = []
        elabels_list = []
        label_list = []
        for idx, (graph, label) in tqdm.tqdm(enumerate(zip(self.graph_dict.values(), self.label_df['target']))):  
            edgeindex, vlabels, elabels = graph2labelledgraph(graph)
            if len(elabels)//2 > self.max_edges:
                continue
            maxdegree = max(maxdegree, max(vlabels))
            edgeindex_list += [edgeindex]
            vlabels_list += [vlabels]
            elabels_list += [elabels]
            label_list += [self.label_df['target'][idx]]
            if maxn is not None:
                if idx >= maxn:
                    break
        self.maxdegree = maxdegree
        
        for idx, (edgeindex, vlabels, elabels, label) in tqdm.tqdm(enumerate(zip(edgeindex_list,
                                                                                vlabels_list,
                                                                                elabels_list, 
                                                                                label_list))):
            g = nx.Graph()
            g.add_edges_from(edgeindex.T.tolist())
            clustering_coefs = nx.triangles(g)
            clustering_coefs = torch.tensor([clustering_coefs[idx] for idx in range(len(clustering_coefs))])
            eccentricity = nx.eccentricity(g)
            eccentricity = torch.tensor([eccentricity[idx] for idx in range(len(eccentricity))], dtype=torch.long)
            
            #centrality_ev = nx.eigenvector_centrality(g, max_iter=1000, tol=1e-4)
            #centrality_deg = nx.degree_centrality(g)
            #centrality_closeness = nx.closeness_centrality(g)
            #centrality_sp = nx.betweenness_centrality(g)
            #centrality_subgraph = nx.subgraph_centrality(g)
            #centrality = []
            #for idx in range(len(centrality_ev)):
            #    centrality += [[centrality_ev[idx],
            #                    centrality_deg[idx],
            #                    centrality_closeness[idx],
            #                    centrality_sp[idx],
            #                    centrality_subgraph[idx]]]
            deg_feats = F.one_hot(torch.tensor(vlabels), num_classes=maxdegree+1).float()
            ecc_feats = F.one_hot(eccentricity, num_classes=self.max_edges).float()
            clust_feats = F.one_hot(eccentricity, num_classes=self.max_edges).float()
            #clust_feats = clustering_coefs.unsqueeze(1)
            node_features = torch.cat((deg_feats, ecc_feats, clust_feats), dim=1)
            edge_features = F.one_hot(torch.tensor(elabels), num_classes=2).float()
            self.data += [Data(**{"edge_index": torch.tensor(edgeindex, dtype=torch.long),
                                "x": node_features, #torch.tensor(centrality),
                                "centrality_deg": node_features,
                                "y": torch.tensor(label, dtype=torch.long),
                                "num_nodes": len(node_features)})]
                                #"centrality_ev": torch.tensor([centrality_ev[idx] for idx in range(len(centrality_ev))])})]

    def len(self):
        return len(self.data)

    def get(self, idx):
        return self.data[idx]
        

In [8]:
dataset = KarateClubDataset(config.graph_file, config.label_file, max_n=config.n_samples, max_edges=config.max_edges)

203088it [00:40, 5020.19it/s]
203088it [04:13, 801.62it/s]


In [9]:
dim_input = dataset[0].x.shape[1]

In [10]:
from einops import rearrange
model = tnn.Sequential('x, edge_index, batch', [
    (eval(config.model)(dim_input, config.n_channels, config.n_layers), 'x, edge_index -> x'),
    (eval(config.readout), 'x, batch -> x'),
    (nn.Linear(config.n_channels, 2),'x -> x')
])

In [11]:
n = len(dataset)
n_train = int(0.8*n)
n_valid = 0
n_test = n - n_train - n_valid
perm = np.random.permutation(len(dataset))
train_idx = torch.tensor(perm[:n_train], dtype=torch.long)
valid_idx = torch.tensor(perm[n_train:n_train+n_valid].tolist(), dtype=torch.long)
test_idx = torch.tensor(perm[n_train+n_valid:].tolist(), dtype=torch.long)

In [12]:
ce = nn.CrossEntropyLoss()
#ce = nn.BCEWithLogitsLoss()

In [13]:
def collate_fn(data):
    return data, data.y

In [14]:
trainloader = DataLoader(dataset, sampler=torch.utils.data.SubsetRandomSampler(train_idx), 
                         batch_size=config.batch_size, collate_fn=collate_fn, num_workers=config.num_workers)
validloader = DataLoader(dataset, sampler=torch.utils.data.SubsetRandomSampler(valid_idx), 
                         batch_size=config.batch_size, num_workers=config.num_workers)
testloader = DataLoader(dataset, sampler=torch.utils.data.SubsetRandomSampler(test_idx), 
                        batch_size=config.batch_size, num_workers=config.num_workers)

In [15]:
def loss(pred, y, ce=ce):
    return ce(pred, y.squeeze())

def auc(pred, y):
    return roc_auc_score(y.detach().cpu().numpy(), pred.detach().cpu().numpy()[:, 1])
    #return roc_auc_score(y.detach().cpu().squeeze().numpy(), pred.detach().cpu().squeeze().numpy())
    

In [16]:
trainer = TrainerGNN(model, trainloader, loss, metrics={'auc': auc}, n_epochs=config.n_epochs, lr=config.learning_rate, validloader=testloader, 
                  es_period=n_train//config.batch_size, lr_adjustment_period=10*n_train//config.batch_size//4, wandb_run=run,
                  clip_gradient_norm=None, lr_patience=5, es_patience=20)

In [ ]:
trainer.fit()

Valid 1: loss 0.486538 0.8969:   3%|████▌                                                                                                                                         | 26/813 [00:03<00:53, 14.74it/s]


Valid 1: loss 0.493220 0.6858:   8%|██████████▋                                                                                                                                   | 61/813 [00:04<00:25, 29.99it/s]


Valid 1: loss 0.497903 0.8173:  11%|███████████████▉                                                                                                                              | 91/813 [00:05<00:20, 35.01it/s]


Valid 1: loss 0.494753 0.8029:  15%|████████████████████▉                                                                                                                        | 121/813 [00:06<00:19, 35.04it/s]


Valid 1: loss 0.492654 0.9032:  19%|██████████████████████████▋                                                                                                                  | 154/813 [00:07<00:19, 33.68it/s]


...

Valid 6: loss 0.493995 0.7603:  96%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎     | 780/813 [00:24<00:00, 36.98it/s]


Valid 6: loss 0.495076 0.8250: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 813/813 [00:26<00:00, 30.97it/s]


Epoch 6: loss 0.493136 0.8384: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3250/3250 [01:53<00:00,  1.32s/it]

EarlyStopping counter: 1 out of 20


  0%|                                                                                                                                                                                      | 0/813 [00:00<?, ?it/s]

## import gc
gc.collect()